In [1]:
import re
import numpy as np
import pandas as pd
import requests,time
import datetime
from scipy import stats
import csv
from bs4 import BeautifulSoup
from requests_html import HTMLSession
import json
import webbrowser
import warnings
import PyPDF2
import io
import requests
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS


In [2]:
ls16_sess = [(pd.read_json(f'16session{i}.json', convert_dates=False)) for i in range(2,18)]
ls17_sess = [(pd.read_json(f'17session{i}.json', convert_dates=False)) for i in range(1,13)]

ls_sess = [*ls16_sess ,*ls17_sess]

# * appending all the sessions to the one dataframe
concatdf = pd.concat(ls_sess, ignore_index=True) # ? only 6 sessions rn

# ? cleaning the data for member names and dates 
# *  Date formatting 
concatdf['date'] = concatdf['date'].str.replace('.','')
concatdf['date'] = (concatdf['date'].str[-4:]+ concatdf['date'].str[2:4] + concatdf['date'].str[:2]).apply(int)  # ? date formating
# *  Member name formatting
concatdf['memberstring'] = [','.join(map(str, l)) for l in concatdf['member']] # ? cleaning the data for member
fourth_column = concatdf.pop('memberstring')
concatdf.insert(4,'memberstring',fourth_column)
concatdf["memberstring"].str.strip()

concatdf["type"].str.strip()
# * _________________________________________________________________________



# concatdf.sort_values(by = 'date')


# * total dates, subjects unique ________________________________________________________________
subjects_unique = concatdf['subjects']
subjects_unique.unique()
print(len(subjects_unique.unique()))
print(len(subjects_unique))
session_dates = concatdf['date']
len(session_dates.unique())

#concatdf
# ? Choose a range of dates 
start_date = 20140707
end_date =  20160303
concatdf['questionText'] = concatdf.questionText.str.replace("\n\r\n","")
#concatdf = concatdf.query(f'date >= {start_date}  & date <= {end_date}') # 20140707 to 20230811
warnings.filterwarnings("ignore",'This pattern is interpreted as a regular expression, and has match groups.To actually get the groups, use str.extract.')
concatdf = concatdf.sort_values(by = 'date', ignore_index= True)

loks16_mem = pd.read_excel("LS16MEMSTATE.xlsx")
loks17_mem = pd.read_excel("LS17MEMSTATE.xlsx")

108948
134866


In [30]:

def ls_frame(start_date, end_date):
   """_returns the datframe with question for the concerned time frame_

    Args:
        start_date (_int_): _the starting date >= 20140707 for now_
        end_date (_type_): _the last date <= 20230811_

    Returns:
        _dataframe_: _with date constraints_
    """   ''''''
   ls_frame = concatdf.query(f'date >= {start_date}  & date <= {end_date}') # 20140707 to 20230811
   return ls_frame


def pdf_indexes(dataframe):
    '''returns indexes of dataframe for column questionsFilePath, links which are pdf files'''
    
    alltr_indexes = []
    allfa_indexes = []

    for i in range(19097,dataframe.index[-1]):          #for word cloud
        val = dataframe.questionsFilePath[i]
        if 'pdf' in val:
            alltr_indexes.append(i)
        else:
            allfa_indexes.append(i)

    return alltr_indexes, allfa_indexes




def subjects_uniqueliststr(dataframe):
    '''Makes all the subjects as string
        for a dataframe, returns a string of all the subjects'''
    subjects_unique = dataframe['ministry']
    subjects_unique_list =  list(subjects_unique)
    subjects_unique_liststr = str(subjects_unique_list)   #for word cloud
    subjects_unique_liststr
    return subjects_unique_liststr




def scrape_pdf_and_questions(dataframe):
  '''Can scrape questions from pdf links in rows after regex confi and paste it on qtext columns
  given a dataframe'''

   #if "pdf" in url:
  indices = pdf_indexes(dataframe)[0]
  for ind in indices:                               # ? could just making a list of strings and pasting them sideways be faster
    url = dataframe.questionsFilePath[ind]
    if "https://" in url:
      r = requests.get(url)
      f = io.BytesIO(r.content)
      reader = PyPDF2.PdfReader(f)
      contents = reader.pages[0].extract_text().split('\n')
      string_contents = str(contents)
      dataframe.loc[ind,'questionText'] = string_contents
      #have to add regular_expressions
    
      #listn = [','.join(map(str, l)) for l in contents]



def total_club_allmem(dataframe):
    """_returns datframe containing questions that were clubbed_

    Args:
        dataframe (_type_): _description_

    Returns:
        _type_: _description_
    """    
    
    names = dataframe.loc[:,["member","memberstring"]]#.reset_index()
    #names = names.drop(["index"], axis = 1)
    clubbed_questions_df = names[names.loc[:,"member"].apply(len) != 1 ]
    return clubbed_questions_df






def  members_who_asked_quest(dataframe):
   ''' returns a list of members who asked questions either independently
   or clubbed, and total number of members who asked questions'''



   lok_sabha_members = [dataframe.loc[i,'member'][0] for i in dataframe.index]
   for i in total_club_allmem(dataframe).index:
    for j in range(len(dataframe.loc[i,'member'])):
        lok_sabha_members.append(dataframe.loc[i,'member'][j])
    
    lok_sabha_members = (set(lok_sabha_members))
    length = len(lok_sabha_members)
    return lok_sabha_members, length
   






def ministry_frequency(dataframe):
    '''returns a dataframe for a given dataframe highlighting, ministries and the amount of questions asked to them with
    respective proportion'''

    ministry_unique = dataframe['ministry']
    ministry_unique = pd.DataFrame(dataframe['ministry'].value_counts())
    ministry_unique['proportion%'] = ((dataframe['ministry'].value_counts())/dataframe['ministry'].shape[0])*100
    ministry_unique.loc['Column_Total']= ministry_unique.sum(numeric_only=True, axis=0)
    ministry_unique.reset_index(inplace=True)
    return ministry_unique





def clubbed_questions_of_a_member(member_name,dataframe):
    """_returns a dataframe containing all the questions of particular member that are clubbed with someone else_

    Args:
        member_name (_string_): _name of the member_
        dataframe (_dataframe_): _the parent dataframe from which you want to see_

    Returns:
        _dataframe_: _Questions that are clubbed the member_
    """    
    member = dataframe.loc[dataframe["memberstring"].str.contains(member_name, case=False)]
    names = member.loc[:,:].reset_index()
    names = names.drop(["index"], axis = 1)
    clubbed_questions_df = names[names.loc[:,"memberstring"] != member_name]
    return clubbed_questions_df



def Questions_Clubbed_with_whom(member,dataframe):
    """_with whom and  the number of times questions got clubbed with_

    Args:
        member (_string_): _member's name acoording to the way in this data_
        dataframe (_dataframe_): _parent dataframe_

    Returns:
        _dataframe_: _containing with all the other members and number of times the member's questions were clubbed with_
    """
    membersask = members_who_asked_quest(dataframe)[0]
    member = clubbed_questions_of_a_member(member,dataframe)
    #supriya = supriya[supriya.memberstring.str.contains("DHANANJAY BHIMRAO MAHADIK", case = False)].shape[0]
   
    club_members = []
    for membs in sorted(membersask):
        count = member[member.memberstring.str.contains(membs, case = False)].shape[0]
        club_mem = {f"MemberName": membs,
                    "Questions_clubbed": count           
        }
        club_members.append(club_mem)
    club_members   = pd.DataFrame.from_dict(club_members)
    return club_members.sort_values(by = "Questions_clubbed", ascending = False)[:].reset_index()


def get_star_unstar_tot_club(member_name,dataframe):
    """_summary_

    Args:
        member_name (_type_): _description_
        dataframe (_type_): _description_

    Returns:
        _dictionary_: _description_
    """    
    member_name = member_name
    #warnings.filterwarnings("ignore", 'This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.')
    the_name = dataframe.loc[dataframe['memberstring'].str.contains(member_name,case = False)]
    clubbed_list = []

    club_count = clubbed_questions_of_a_member(member_name,dataframe).shape[0]


    unstarred = 0 if len(the_name.type.value_counts()) < 1 else the_name.type.value_counts().iloc[0]
    starred   = 0 if len(the_name.type.value_counts()) < 2 else the_name.type.value_counts().iloc[1] 
    dictionary_name_ques = {'MemberName' :f'{member_name}',
                            'Total_Questions' : unstarred+starred,
                            'Clubbed': club_count,
                            'Unstarred': unstarred,
                            'Starred' : starred,
                            '% Starred' : 1+starred/(1+unstarred+starred) -1

    }
    return dictionary_name_ques





def compdf_with_states(dataframe,lsno_member_list_states):
    '''comparison df goes into this, with cleaned member_list(second parameter) of states and party'''
    dataframe["State"] = "N/A"
    dataframe["Party Name"] = "N/A"
    for i in range(dataframe.shape[0]):
        ser = lsno_member_list_states.loc[:,"Name of Member"].str.contains(dataframe.loc[i,"MemberName"],case= False,)
        
        if len(ser[ser].index) != 0:
            dataframe.loc[i,"State"] = str(lsno_member_list_states.loc[ser[ser].index[0],"State"])
            dataframe.loc[i,"Party Name"] = str(lsno_member_list_states.loc[ser[ser].index[0],"Party Name"])
        
    return dataframe





list = ['a','b']
def comparison_df(dataframe,lsno_member_list_states):
    """_returns a dataframe for a given dataframe with members total_questions asked prop of 
   clubbed, starred and unstarred_

    Args:
        dataframe (_type_): _description_
        lsno_member_list_states (_type_): _description_

    Returns:
        _type_: _description_
    """
    data = []
    for member_name in members_who_asked_quest(dataframe)[0]:
        data.append(get_star_unstar_tot_club(member_name,dataframe))
        comparison_df = pd.DataFrame.from_dict(data)
        comparison_df.MemberName = comparison_df.MemberName.replace(r'\s+', ' ', regex=True)
    return compdf_with_states(comparison_df.sort_values(by = "Starred", ascending = False, ignore_index= True),lsno_member_list_states)







def question_subject2mem(mem_list,dataframe):
    """_What subjects or ministry to questions were clubbed with a particular member_

    Args:
        mem_list (_list_): _list containg the primary filter name at the 0 index_
        dataframe (_dataframe_): _the parent dataframe_

    Returns:
        _dataframe_: _returning the instances of the members in the list asking clubbed questions or part of another_
    """     
    mem = dataframe.loc[dataframe['memberstring'].str.contains(mem_list[0], case= False)]
    for i in range(1,len(mem_list)):
        mem = mem.loc[mem['memberstring'].str.contains(mem_list[i], case = False)].reset_index().drop("index", axis=1)
    return mem


















What a member of a certain party clubs with another member of another party
    
1. So what type of questions, ministries  Supriya sule clubs with Eknath shinde?
2. Cleaning the warnings
3. Data plotting

In [6]:
ls_20142016 = ls_frame(20140707,20160303)
ls_20142016

,quesNo,subjects,lokNo,member,memberstring,ministry,type,date,questionText,answerText,questionsFilePath,questionsFilePathHindi,sessionNo,supplementaryQuestionResDtoList,supplementaryType
0,65,RAISING RETIREMENT AGE IN SICK CPSES,16,[FAIZAL P.P. MOHAMMED],FAIZAL P.P. MOHAMMED,HEAVY INDUSTRIES AND PUBLIC ENTERPRISES,UNSTARRED,20140707,(a) whether the Government proposes or has any...,\r\nTHE MINISTER OF STATE FOR HEAVY INDUSTRIES...,,NaN,2,NaN,False
1,63,MOBILE COMMUNICATION SYSTEM,16,[ARJUN RAM MEGHWAL],ARJUN RAM MEGHWAL,COMMUNICATIONS AND INFORMATION TECHNOLOGY,UNSTARRED,20140707,(a) whether the Government has taken note of ...,\r\n\n\nTHE MINISTER OF COMMUNICATIONS AND INF...,,NaN,2,NaN,False
2,62,DEMAND AND SUPPLY OF NATURAL GAS,16,[NALIN KUMAR KATEEL],NALIN KUMAR KATEEL,PETROLEUM AND NATURAL GAS,UNSTARRED,20140707,\r\n(a)\tthe domestic demand and supply of nat...,\r\n\n\nMINISTER OF STATE (I/C) IN THE MINISTR...,,NaN,2,NaN,False
3,61,REFORMS IN CRIMINAL LAW,16,[HANSRAJ GANGARAM AHIR],HANSRAJ GANGARAM AHIR,LAW AND JUSTICE,UNSTARRED,20140707,(a) whether the Government has asked the Law ...,\r\n\n\nMINISTER OF LAW AND JUSTICE & COMMUNIC...,,NaN,2,NaN,False
4,60,BUILDING FACILITIES FOR POST OFFICES,16,[RANGASWAMY DHRUVANARAYANA],RANGASWAMY DHRUVANARAYANA,COMMUNICATIONS AND INFORMATION TECHNOLOGY,UNSTARRED,20140707,(a) whether the Government has any proposal t...,\r\n\n\nTHE MINISTER OF COMMUNICATIONS AND INF...,,NaN,2,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29542,1233,Decline in Water Level of Wells,16,[PRATHAP SIMHA],PRATHAP SIMHA,"WATER RESOURCES, RIVER DEVELOPMENT AND GANAGA ...",UNSTARRED,20160303,NaN,NaN,https://sansad.in/getFile/loksabhaquestions/an...,https://sansad.in/getFile/loksabhaquestions/qh...,7,NaN,False
29543,1232,Inter-State River Water Sharing Disputes,16,"[BAHADUR SINGH KOLI, CH. MALLA REDDY, DUSHYANT...","BAHADUR SINGH KOLI,CH. MALLA REDDY,DUSHYANT SI...","WATER RESOURCES, RIVER DEVELOPMENT AND GANAGA ...",UNSTARRED,20160303,NaN,NaN,https://sansad.in/getFile/loksabhaquestions/an...,https://sansad.in/getFile/loksabhaquestions/qh...,7,NaN,False
29544,1231,PMSY for MGNREGA Workers,16,[VINOD KUMAR SONKAR],VINOD KUMAR SONKAR,RURAL DEVELOPMENT,UNSTARRED,20160303,NaN,NaN,https://sansad.in/getFile/loksabhaquestions/an...,https://sansad.in/getFile/loksabhaquestions/qh...,7,NaN,False
29545,1230,Research in Water Management,16,[KUNWAR PUSHPENDRA SINGH CHANDEL],KUNWAR PUSHPENDRA SINGH CHANDEL,"WATER RESOURCES, RIVER DEVELOPMENT AND GANAGA ...",UNSTARRED,20160303,NaN,NaN,https://sansad.in/getFile/loksabhaquestions/an...,https://sansad.in/getFile/loksabhaquestions/qh...,7,NaN,False


In [21]:

top_sttared = comparison_df(ls_20142016,loks16_mem)


C:\Users\shubh\AppData\Local\Temp\ipykernel_17964\909254261.py:101: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  the_name = dataframe.loc[dataframe['memberstring'].str.contains(member_name,case = False)]
C:\Users\shubh\AppData\Local\Temp\ipykernel_17964\909254261.py:75: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  member = dataframe.loc[dataframe["memberstring"].str.contains(member_name, case=False)]
C:\Users\shubh\AppData\Local\Temp\ipykernel_17964\909254261.py:101: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  the_name = dataframe.loc[dataframe['memberstring'].str.contains(member_name,case = False)]
C:\Users\shubh\AppData\Local\Temp\ipykernel_17964\909254261.py:75: UserWarning: This pattern is interpreted as a regular e

In [24]:
top_sttared

,MemberName,Total_Questions,Clubbed,Unstarred,Starred,% Starred,State,Party Name
0,DHARMENDRA YADAV,428,411,386,42,0.097902,Uttar Pradesh,Samajwadi Party
1,SUPRIYA SADANAND SULE,483,454,444,39,0.080579,Maharashtra,Nationalist Congress Party
2,NISHIKANT DUBEY,339,217,303,36,0.105882,Jharkhand,Bharatiya Janata Party
3,RAJEEV SHANKARRAO SATAV,434,422,401,33,0.075862,Maharashtra,Indian National Congress
4,DHANANJAY BHIMRAO MAHADIK,465,462,433,32,0.068670,Maharashtra,Nationalist Congress Party
...,...,...,...,...,...,...,...,...
454,AKSHAY YADAV,1,0,1,0,0.000000,Uttar Pradesh,Samajwadi Party
455,(SMT.) RATNA DE(NAG),0,0,0,0,0.000000,N/A,N/A
456,ASRARUL HAQUE MOHAMMAD,1,0,1,0,0.000000,Bihar,Indian National Congress
457,AJAY (TENI) MISRA,0,0,0,0,0.000000,N/A,N/A


In [28]:
i = "DHANANJAY BHIMRAO MAHADIK"
supriya = clubbed_questions_of_a_member("SUPRIYA SADANAND SULE",ls_20142016)
count = supriya[supriya.memberstring.str.contains(i, case = False)].shape[0]
count
membernames = "SUPRIYA SADANAND SULE"
#member = ls_20142016.loc[ls_20142016["memberstring"].str.contains(r""+membernames+"", case=False)]
member = ls_20142016.loc[ls_20142016["memberstring"].str.contains(membernames, case=False)]
member

,quesNo,subjects,lokNo,member,memberstring,ministry,type,date,questionText,answerText,questionsFilePath,questionsFilePathHindi,sessionNo,supplementaryQuestionResDtoList,supplementaryType
22,2,EMPLOYMENT GENERATION,16,[SUPRIYA SADANAND SULE],SUPRIYA SADANAND SULE,LABOUR AND EMPLOYMENT,STARRED,20140707,\r(a)\twhether the Government has taken note o...,"\r\n\n\nMINISTER OF STEEL, MINES, LABOUR AND E...",https://sansad.in/getFile/Annexture_New/lsq16/...,NaN,2,NaN,True
43,53,FUNCTIONING OF EMPLOYMENT EXCHANGES,16,"[SUPRIYA SADANAND SULE, VIJAYSINH MOHITE PATIL]","SUPRIYA SADANAND SULE,VIJAYSINH MOHITE PATIL",LABOUR AND EMPLOYMENT,UNSTARRED,20140707,(a)\tthe total number of Employment Exchanges ...,"\r\n\n\nMINISTER OF STATE FOR STEEL, MINES, LA...",https://sansad.in/getFile/Annexture_New/lsq16/...,NaN,2,NaN,False
48,16,SHORTAGE OF CABIN CREW,16,"[SUPRIYA SADANAND SULE, VIJAYSINH MOHITE PATIL...","SUPRIYA SADANAND SULE,VIJAYSINH MOHITE PATIL,D...",CIVIL AVIATION,UNSTARRED,20140707,\r(a) whether there is a shortage of cabin cre...,\nMinister of State in the Ministry of CIVIL A...,,NaN,2,NaN,False
70,37,AMENDMENT TO FACTORIES ACT,16,"[SUPRIYA SADANAND SULE, VIJAYSINH MOHITE PATIL...","SUPRIYA SADANAND SULE,VIJAYSINH MOHITE PATIL,D...",LABOUR AND EMPLOYMENT,UNSTARRED,20140707,(a)whether a small percentage of workforce in ...,"\r\n\n\nMINISTER OF STATE FOR STEEL, MINES, LA...",,NaN,2,NaN,False
107,110,FOREIGN TOURISTS INFLOW,16,[SUPRIYA SADANAND SULE],SUPRIYA SADANAND SULE,TOURISM,UNSTARRED,20140708,(a) whether it is a fact that several countrie...,\r\n\n\nMINISTER OF STATE FOR TOURISM (INDEPEN...,,NaN,2,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29323,1334,Shortage of Pilots,16,"[HEENA VIJAYKUMAR GAVIT, DHARMENDRA YADAV, DHA...","HEENA VIJAYKUMAR GAVIT,DHARMENDRA YADAV,DHANAN...",CIVIL AVIATION,UNSTARRED,20160303,NaN,NaN,https://sansad.in/getFile/loksabhaquestions/an...,https://sansad.in/getFile/loksabhaquestions/qh...,7,NaN,False
29342,1337,Grid Connected Rooftop Solar Panels,16,"[Richard Hay, DHANANJAY BHIMRAO MAHADIK, JAGDA...","Richard Hay,DHANANJAY BHIMRAO MAHADIK,JAGDAMBI...",NEW AND RENEWABLE ENERGY,UNSTARRED,20160303,NaN,NaN,https://sansad.in/getFile/loksabhaquestions/an...,https://sansad.in/getFile/loksabhaquestions/qh...,7,NaN,False
29359,1338,Delay in Work on NH-9,16,"[VIJAYSINH MOHITE PATIL, DHANANJAY BHIMRAO MAH...","VIJAYSINH MOHITE PATIL,DHANANJAY BHIMRAO MAHAD...",ROAD TRANSPORT AND HIGHWAYS,UNSTARRED,20160303,NaN,NaN,https://sansad.in/getFile/loksabhaquestions/an...,https://sansad.in/getFile/loksabhaquestions/qh...,7,NaN,False
29405,1339,Highway Advisory Services,16,"[BHARATENDRA SINGH, SUPRIYA SADANAND SULE, VIJ...","BHARATENDRA SINGH,SUPRIYA SADANAND SULE,VIJAYS...",ROAD TRANSPORT AND HIGHWAYS,UNSTARRED,20160303,NaN,NaN,https://sansad.in/getFile/loksabhaquestions/an...,https://sansad.in/getFile/loksabhaquestions/qh...,7,NaN,False


In [8]:
top_starred20 = top_sttared[:20]
top_starred = compdf_with_states(top_starred,loks16_mem)
top_starred.reset_index().drop("index", axis=1)
(top_starred20.loc[:,["State"]] == "Maharashtra").value_counts()


NameError: name 'top_starred' is not defined

In [ ]:
ministry_frequency(ls_20142016)

In [7]:
top_starred20



NameError: name 'top_starred20' is not defined